In [2]:
import pandas as pd
import wandb
import os 
import sys 
__file__ = 'wandb_test.ipynd'
os.path.abspath(os.path.join(os.path.dirname(__file__), '..'))
api = wandb.Api()
entity, project = "tpn", "rllib1"
runs = api.runs(entity + "/" + project)
RESULTS_DIR = os.path.join(os.path.dirname(__file__), '..', 'results')
ALGORITHM_TYPES = ["independent", "centralized", "shared"]
REWARD_TYPES = ["type_1", "type_2", "type_3"]
STEP_PENALTY = [0, 0.01, 0.03]
ALGORITHM_TYPE = "independent"
REWARD_TYPE = "type_1"

This cell prints the performances of the agents in the file > results/stats_PROJECT_NAME_RUNS.txt

In [4]:
_run_stats = dict()
collated_performances_df = pd.DataFrame(
    [],
    columns = ['step_penalty', 'algorithm_type', 'reward_type', 'lstm',
        'episode_length', 'episode_reward', 'epsiode_assists',\
               'eval_length', 'eval_reward', 'eval_assists']
).fillna(0)


for run in runs:
    # if run._state != "finished":
    if run._state != "finished":
        if run._state == "running":
            _run_stats['running_runs'] = _run_stats.get('running_runs', 0) + 1
        else:
            _run_stats['crashed_runs'] = _run_stats.get('crashed_runs', 0) + 1
        continue
    _run_stats['finished_runs'] = _run_stats.get('finished_runs', 0) + 1

    config = run.config
    algo_type = config['algorithm_type']
    reward_type = config['env_config']['reward_type']
    step_penalty = config['env_config']['step_penalty']
    use_lstm = config['training']['model']['use_lstm']
    collated_performances_df.loc[len(collated_performances_df)] = [
        step_penalty,
        algo_type,
        reward_type,
        use_lstm,
        run.summary['episode_len_mean'],
        run.summary['episode_reward_mean'],
        run.summary['episode_assists_mean'],
        run.summary['eval_episode_len'],
        run.summary['eval_reward'],
        run.summary['eval_assists']
    ]

print(f"F: {_run_stats.get('finished_runs', 0)}, R: {_run_stats.get('running_runs', 0)}, C: {_run_stats.get('crashed_runs', 0)}")
collated_performances_df = collated_performances_df.groupby(['step_penalty', 'reward_type', 'lstm', 'algorithm_type']).agg(['mean', 'std', 'count'])
with open(f"{RESULTS_DIR}/stats_{project}_{_run_stats.get('finished_runs')}.txt", 'a') as f:
    f.write(f"Finished: {_run_stats.get('finished_runs', 0)}, Running: {_run_stats.get('running_runs', 0)}, Crashed: {_run_stats.get('crashed_runs', 0)}\n")
    f.write(collated_performances_df.to_string())
    f.write("\n\n\n")
    # f.write(collated_performances_df.to_latex())
    # f.write("\n\n\n")
    # f.write(collated_performances_df.to_markdown())
    # f.write("\n\n\n")
collated_performances_df
    

F: 410, R: 0, C: 98


episode_length                   \
                                                        mean        std count   
step_penalty reward_type lstm  algorithm_type                                   
-0.03        type_1      False centralized         67.105000   7.987076    10   
                               independent         68.789000   6.543206    10   
                               shared              67.593636   3.555897    11   
                         True  centralized         78.162500  14.230646     8   
                               independent         61.259167   5.698862    12   
                               shared              65.864167  10.715221    12   
             type_2      False centralized         69.087778   8.175327     9   
                               independent         71.448182   7.236193    11   
                               shared              71.634545   4.017022    11   
                         True  centralized         80.846667   7.096052     9   
                               independent         64.564545   4.077176    11   
                               shared              69.939167   7.951358    12   
-0.01        type_1      False centralized         60.021818   5.834715    11   
                               independent         60.033333   4.947115    12   
                               shared              65.590833   4.399858    12   
                         True  centralized         71.121818  11.649911    11   
                               independent         55.407500   3.769825    12   
                               shared              61.790000   6.064007    12   
             type_2      False centralized         58.313000   5.341677    10   
                               independent         59.150000   4.808403    11   
                               shared              69.496667   3.861550    12   
                         True  centralized         70.678889   9.758911     9   
                               independent         59.151667   7.728423    12   
                               shared              65.401538   4.886314    13   
 0.00        type_1      False centralized         54.252500   3.249624    12   
                               independent         53.753333   2.217912    12   
                               shared              63.995833   3.197472    12   
                         True  centralized         67.277500  10.184834    12   
                               independent         57.918333   7.194155    12   
                               shared              60.502857   9.266436    14   
             type_2      False centralized         56.395833   5.664568    12   
                               independent         54.205000   5.206675    12   
                               shared              66.543333   2.611089    12   
                         True  centralized         71.469091  13.448919    11   
                               independent         56.795385   6.736073    13   
                               shared              60.652308   4.775047    13   

                                              episode_reward                  \
                                                        mean       std count   
step_penalty reward_type lstm  algorithm_type                                  
-0.03        type_1      False centralized          9.530900  0.323426    10   
                               independent          9.643300  0.244340    10   
                               shared               9.522345  0.146322    11   
                         True  centralized          9.821262  0.478052     8   
                               independent          9.287358  0.254338    12   
                               shared               9.286917  0.197920    12   
             type_2      False centralized          9.826800  0.313573     9   
                               independent          9.971927  0.287329    11   
      

This cell prints the collated analysis df for each group [step_penalty, algo_type, reward_type] and writes them in files > [r]-[algo_type]_[reward_type]_[step_penalty]

In [39]:
import re


def get_analysis_df(algo_type, reward_type, step_penalty, use_lstm):
    file_name_prefix = "" # _chaser, _fixed, _follower
    _run_stats = dict()
    big_analysis_df = None
    run_history_df = pd.DataFrame(
        [],
            columns = [
            'run_name',
            'trial_count',
            'training_iteration',
            'episodes_total',
            'train_len',
            'train_reward',
            'train_assists',
            'eval_len',
            'eval_reward',
            'eval_assists',
            'analysis_len',
            'analysis_reward',
            'analysis_assists',]
    ).fillna(0) 
    eval_df = None
    for run in runs:
        if run._state != "finished":
            if run._state == "running":
                _run_stats['running_runs'] = _run_stats.get('running', 0) + 1
            else:
                _run_stats['crashed_runs'] = _run_stats.get('crashed', 0) + 1
            continue
        _run_stats['finished_runs'] = _run_stats.get('finished', 0) + 1

        config = run.config
        if config['algorithm_type'] == algo_type and \
            config['env_config']['reward_type'] == reward_type and \
                config['env_config']['step_penalty'] == step_penalty and \
                    config['training']['model']['use_lstm'] == use_lstm:
            _run_stats['matching_runs'] = _run_stats.get('matching_runs', 0) + 1

            if file_name_prefix == "":
                file_name_prefix = algo_type 

            for file in run.files():
                if f"{file_name_prefix}_analysis" in file.name:
                    analysis_df = pd.read_json(
                        file.download(exist_ok=True), 
                        orient='split')
                    for file in run.files():
                        if f"{file_name_prefix}_eval" in file.name:
                            eval_stats = pd.read_json(
                                file.download(exist_ok=True), 
                                orient='split')
                            break
                    col_list = analysis_df.groupby(['mode', 'agent_a', 'agent_b', 'test', 'dimension']).count().columns
                    agg_funcs = []
                    for col in col_list:
                        if col == 'run_id':
                            agg_funcs.append((col, 'nunique'))
                        else:
                            agg_funcs.append((col, 'mean'))
                    analysis_df = analysis_df.groupby(['mode', 'agent_a', 'agent_b', 'test', 'dimension']).agg(
                        dict(agg_funcs)
                    ).reset_index()
                    analysis_df['trial_count'] = analysis_df['run_id']
                    analysis_df.drop(columns=['run_id'], inplace=True)

                    big_analysis_df = pd.concat([big_analysis_df, analysis_df]) if big_analysis_df is not None else analysis_df

                    run_history_df.loc[len(run_history_df)] = [
                        run.name,
                        analysis_df['trial_count'][0],
                        run.summary['training_iteration'],
                        run.summary['episodes_total'],
                        run.summary['episode_len_mean'],
                        run.summary['episode_reward_mean'],
                        run.summary['episode_assists_mean'],
                        run.summary['eval_episode_len'],
                        run.summary['eval_reward'],
                        run.summary['eval_assists'],
                        eval_stats['episode_len'].mean(),
                        eval_stats['episode_reward'].mean(),
                        eval_stats['assists'].mean()]
    if len(run_history_df) == 0:
        return None, None, _run_stats
    
    col_list = big_analysis_df.groupby(['mode', 'agent_a', 'agent_b', 'test', 'dimension']).count().columns
    agg_funcs = []
    for col in col_list:
        if col == 'run_name':
            agg_funcs.append((col, 'nunique'))
        elif col == 'trial_count':
            agg_funcs.append((col, 'sum'))
            agg_funcs.append((col, 'nunique'))
        else:
            agg_funcs.append((col, 'mean'))
    big_analysis_df = big_analysis_df.groupby(['mode', 'agent_a', 'agent_b', 'test', 'dimension']).agg(
        dict(agg_funcs)
    )
    run_history_df.set_index('run_name', inplace=True)  
    return big_analysis_df, run_history_df, _run_stats
                
for algo_type in ['independent', 'centralized', 'shared']:
    for reward_type in ['type_1', 'type_2', 'type_3']:
        for step_penalty in [0, 0.01, 0.03]:
            for use_lstm in [True, False]:
                analysis_df, run_history_df, _run_stats = get_analysis_df(
                algo_type,
                reward_type,
                step_penalty,
                use_lstm)
                print(f"{algo_type}, {reward_type}, {step_penalty}: {_run_stats}")

                if analysis_df is None:
                    continue

                file_name = f"{algo_type}_{reward_type}_{step_penalty}"
                if use_lstm:
                    file_name  = "r-" + file_name
                file_name = project+"_"+file_name +".txt"
                with open(f"{RESULTS_DIR}/{file_name}", 'w') as f:
                    descr = f"""{algo_type}, {reward_type}, {step_penalty}: {_run_stats}
                    COMPLETED RUNS: {_run_stats.get('finished_runs', 0)}
                    RUNNING RUNS: {_run_stats.get('running_runs', 0)}
                    CRASHED RUNS: {_run_stats.get('crashed_runs', 0)}
                    TOTAL = {_run_stats.get('finished_runs', 0) + _run_stats.get('running_runs', 0) + _run_stats.get('crashed_runs', 0)}
                    \nBASE PERFORMANCE: {run_history_df.mean()}
                    """
                    print(descr)
                    f.write(descr)
                    f.write(run_history_df.to_string())
                #     f.write("\n\n\n")
                #     f.write(run_history_df.to_latex())
                #     f.write("\n\n\n")
                #     f.write(run_history_df.to_markdown())
                #     f.write("\n\n\n")
 

independent, type_1, 0: {'crashed_runs': 1, 'finished_runs': 1, 'matching_runs': 12}
independent, type_1, 0: {'crashed_runs': 1, 'finished_runs': 1, 'matching_runs': 12}
                    COMPLETED RUNS: 1
                    RUNNING RUNS: 0
                    CRASHED RUNS: 1
                    TOTAL = 2
                    
BASE PERFORMANCE: trial_count               2.000000
training_iteration    10775.000000
episodes_total        24928.500000
train_len                63.520000
train_reward              5.810000
train_assists             0.320000
eval_len                 56.514049
eval_reward               5.890166
eval_assists              0.337803
analysis_len             57.716349
analysis_reward           5.838482
analysis_assists          0.310440
dtype: float64
                    
independent, type_1, 0: {'crashed_runs': 1, 'finished_runs': 1, 'matching_runs': 12}
independent, type_1, 0: {'crashed_runs': 1, 'finished_runs': 1, 'matching_runs': 12}
                    COMPL

In [14]:
df1 = run_history_df.set_index('run_name')
df1.mean()

trial_count               2.000000
training_iteration    10629.166667
episodes_total        24939.000000
train_len                60.908333
train_reward              7.451600
train_assists             0.305833
eval_len                 60.920178
eval_reward               7.400486
eval_assists              0.340501
analysis_len             60.551788
analysis_reward           7.465671
analysis_assists          0.330132
dtype: float64

In [41]:
# simple solution that works; use it to verfiy results 
file_name_prefix = "" # _chaser, _fixed, _follower
_run_stats = dict()
collated_df = None
eval_df = None
for run in runs:
    # if run._state != "finished":
    if run._state != "finished":
        if run._state == "running":
            _run_stats['running_runs'] = _run_stats.get('running', 0) + 1
        else:
            _run_stats['crashed_runs'] = _run_stats.get('crashed', 0) + 1
        continue
    _run_stats['finished_runs'] = _run_stats.get('finished', 0) + 1

    config = run.config
    if config['algorithm_type'] == ALGORITHM_TYPE and \
        config['env_config']['reward_type'] == REWARD_TYPE:
        _run_stats['matching_runs'] = _run_stats.get('matching_runs', 0) + 1

        if file_name_prefix == "":
            file_name_prefix = ALGORITHM_TYPE

        for file in run.files():
            if f"{file_name_prefix}_analysis" in file.name:
                analysis_table = pd.read_json(
                    file.download(exist_ok=True), 
                    orient='split')
                if collated_df is None:
                    collated_df = analysis_table
                else:
                    collated_df = pd.concat([collated_df, analysis_table])
            elif f"{file_name_prefix}_eval_df" in file.name:
                eval_stats = pd.read_json(
                    file.download(exist_ok=True), 
                    orient='split')
                if eval_df is None:
                    eval_df = eval_stats
                else:
                    eval_df = pd.concat([eval_df, eval_stats])
grouped_by_runs = collated_df.groupby(['mode', 'agent_a', 'test', 'dimension'])
grouped_by_runs[[str(i) for i in range(500, 5001, 500)]].agg(['mean', 'std', 'count'])

500                      1000  \
                                            mean       std count      mean   
mode        agent_a    test  dimension                                       
independent predator_0 ccm_1 PCA_1      0.361708  0.080423    24  0.366292   
                             PCA_2      0.273875  0.113499    24  0.282250   
                             dx         0.201083  0.080183    24  0.129083   
                             dy         0.172958  0.074315    24  0.107000   
            predator_1 ccm_1 PCA_1      0.348250  0.095791    24  0.362000   
                             PCA_2      0.309375  0.069361    24  0.317333   
                             dx         0.181792  0.083877    24  0.104500   
                             dy         0.163542  0.077954    24  0.107167   

                                                            1500            \
                                             std count      mean       std   
mode        agent_a    test  dimension                                       
independent predator_0 ccm_1 PCA_1      0.075800    24  0.363375  0.063518   
                             PCA_2      0.088054    24  0.290417  0.081881   
                             dx         0.047252    24  0.091708  0.052815   
                             dy         0.054154    24  0.089667  0.061059   
            predator_1 ccm_1 PCA_1      0.080716    24  0.363000  0.077217   
                             PCA_2      0.053196    24  0.312625  0.049518   
                             dx         0.039007    24  0.091583  0.029478   
                             dy         0.049016    24  0.066292  0.050837   

                                                  2000  ...  3500      4000  \
                                       count      mean  ... count      mean   
mode        agent_a    test  dimension                  ...                   
independent predator_0 ccm_1 PCA_1        24  0.352708  ...    24  0.350917   
                             PCA_2        24  0.293333  ...    24  0.287125   
                             dx           24  0.076542  ...    24  0.041542   
                             dy           24  0.063958  ...    24  0.042875   
            predator_1 ccm_1 PCA_1        24  0.354208  ...    24  0.354792   
                             PCA_2        24  0.311250  ...    24  0.315167   
                             dx           24  0.075375  ...    24  0.037208   
                             dy           24  0.062042  ...    24  0.047750   

                                                            4500            \
                                             std count      mean       std   
mode        agent_a    test  dimension                                       
independent predator_0 ccm_1 PCA_1      0.058052    24  0.354708  0.057910   
                             PCA_2      0.069369    24  0.287458  0.065974   
                             dx         0.031129    24  0.032708  0.027836   
                             dy         0.032030    24  0.035792  0.030777   
            predator_1 ccm_1 PCA_1      0.072904    24  0.351750  0.071456   
                             PCA_2      0.047474    24  0.315333  0.048226   
                             dx         0.024546    24  0.043375  0.032430   
                             dy         0.036738    24  0.040958  0.033821   

                                                  5000                  
                                       count      mean       std count  
mode        agent_a    test  dimension                                  
independent predator_0 ccm_1 PCA_1        24  0.353500  0.058284    24  
                             PCA_2        24  0.288667  0.063346    24  
                             dx           24  0.036167  0.023168    24  
                             dy           24  0.044833  0.035768    24  
            predator_1 ccm_1 PCA_1        24  0.354125  0.070428    24  
              

In [11]:

POLICY_SETS = ["chaser_chaser", "chaser_fixed", "chaser_follower", "fixed_follower", "follower_chaser"]
grouped_dfs = []
eval_dfs = None
for policy_set in POLICY_SETS:
    analyis_df = pd.read_csv(f"../experiments/results/{policy_set}_analysis.csv")
    eval_df = pd.read_csv(f"../experiments/results/{policy_set}_eval.csv")
    col_list = list(analyis_df.groupby(['mode', 'agent_a', 'agent_b', 'test', 'dimension']).mean().columns)
    agg_funs = []
    for col in col_list:
        if col != 'run_id':
            agg_funs.append((col, lambda x: round(x.mean(), 2)))
        else:
            agg_funs.append((col, 'count'))
        
    df2 = analyis_df.groupby(['mode', 'agent_a', 'agent_b', 'test', 'dimension']).agg(dict(agg_funs))
    grouped_dfs.append(df2) 
    eval_dfs = pd.concat([
        eval_dfs,
        pd.DataFrame([dict(name=policy_set,**dict(eval_df.mean()))])
    ]) if eval_dfs is not None else pd.DataFrame([dict(name=policy_set,**dict(eval_df.mean()))])
eval_dfs.set_index('name', inplace=True)
with open("1_fixed_analysis.txt", "w") as f:
    f.write("EVALUATION simple map size 15, reward type 1, step penalty None\n")
    f.write(eval_dfs.to_string())
    f.write("\n\n")
    for df in grouped_dfs:
        f.write(df.to_string())
        f.write("\n\n")

In [22]:
df2

run_id       500  \
mode          agent_a    agent_b    test  dimension                     
chaser_chaser predator_0 predator_1 ccm_1 PCA_1           3  0.127000   
                                          PCA_2           3  0.110000   
                                          PCA_3           3  0.249000   
                                          dx              3  0.235333   
                                          dy              3  0.217333   
                                          x               3  0.512333   
                                          y               3  0.521000   
              predator_1 predator_0 ccm_1 PCA_1           3  0.189000   
                                          PCA_2           3  0.162667   
                                          PCA_3           3  0.327667   
                                          dx              3  0.248667   
                                          dy              3  0.251000   
                                          x               3  0.528000   
                                          y               3  0.610333   

                                                         1000      1500  \
mode          agent_a    agent_b    test  dimension                       
chaser_chaser predator_0 predator_1 ccm_1 PCA_1      0.125667  0.131000   
                                          PCA_2      0.090333  0.107000   
                                          PCA_3      0.177333  0.155000   
                                          dx         0.175667  0.108333   
                                          dy         0.181333  0.112667   
                                          x          0.440333  0.398667   
                                          y          0.475667  0.425667   
              predator_1 predator_0 ccm_1 PCA_1      0.210333  0.178333   
                                          PCA_2      0.167333  0.148000   
                                          PCA_3      0.221667  0.187000   
                                          dx         0.151667  0.108333   
                                          dy         0.132000  0.109667   
                                          x          0.459667  0.368667   
                                          y          0.520667  0.447000   

                                                         2000      2500  \
mode          agent_a    agent_b    test  dimension                       
chaser_chaser predator_0 predator_1 ccm_1 PCA_1      0.137667  0.131000   
                                          PCA_2      0.111667  0.138667   
                                          PCA_3      0.159333  0.159667   
                                          dx         0.078333  0.059333   
                                          dy         0.078667  0.065000   
                                          x          0.325333  0.291333   
                                          y          0.363667  0.313333   
              predator_1 predator_0 ccm_1 PCA_1      0.171333  0.170667   
                                          PCA_2      0.152000  0.152333   
                                          PCA_3      0.125333  0.120667   
                                          dx         0.099667  0.060333   
                                          dy         0.073000  0.071667   
                                          x          0.343667  0.308000   
                                          y          0.359667  0.358333   

                                                         3000      3500  \
mode          agent_a    agent_b    test  dimension                       
chaser_chaser predator_0 predator_1 ccm_1 PCA_1      0.142333  0.143000   
                                          PCA_2      0.131000  0.129000   
                                          PCA_3      0.157333  0.153667   
                                          dx         0.066667  0.025667   
                                